In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("./train.csv")

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
# assuming features for now, look into feature engineering for other projects

df = df[["Pclass", "Sex", "Age", "Fare", "Survived"]]

In [ ]:
df.head()

In [ ]:
df.info() # have some null values for age, will fill them with the mean of each feature

In [ ]:
mean_age = df['Age'].mean()
df.fillna(mean_age,axis=1,inplace=True)

df.head()

In [ ]:
#converting sex to nominal values
df['Sex'] = df['Sex'].map({"male":0, "female":1})
df.head()

In [ ]:
x = df[['Pclass', 'Sex', 'Age', 'Fare']].values 
y = df['Survived'].values

In [ ]:
sc = StandardScaler()
x = sc.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .2) # 80% of data set will be used for training

In [ ]:
#converting to tensors
x_train, x_test = torch.tensor(x_train, dtype = torch.float32), torch.tensor(x_test, dtype = torch.float32)
y_train, y_test = torch.tensor(y_train,  dtype = torch.float32), torch.tensor(y_test,  dtype = torch.float32)

In [ ]:
y_train = y_train.view(-1,1)

y_train.shape

input_size = 4 # "Pclass", "Sex", "Age", "Fare"
output_size = 1 # 'Survived'

In [ ]:
model = nn.Sequential(nn.Linear(input_size, output_size),nn.Sigmoid())

In [ ]:
#hyper-paramters
learning_rate = .01
iters = 10000

criterion = nn.BCELoss() #binary classification loss
optimizer = optim.SGD(model.parameters(), lr = learning_rate) #stochastic gradient descent


In [ ]:
for epoch in range(iters):
  y_pred = model(x_train)
  loss = criterion(y_pred, y_train)
  loss.backward() #backpropogation
  optimizer.step()
  optimizer.zero_grad()

  if epoch % 100 == 0: 
    print("Epoch: {}, loss: {}".format(epoch, loss))

In [ ]:
pred = model(x_test).round()

In [ ]:
# accuracy of model 
print(accuracy_score(y_test.detach().numpy(), pred.detach().numpy() ))